In [3]:
import socket
import yaml
import argparse
from protobuf_to_dict import protobuf_to_dict, dict_to_protobuf

from ecat_repl import gen_cmds
from ecat_repl import repl_cmd
from ecat_repl import ZmsgIO

uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
io = ZmsgIO(uri)

id = 1
ids = [id]

homing = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "HOMING", "name": "homing", "board_id": 1,
                             "homing_par": {"x": [0,3]}}}

xs = [0,1,2,3,4]
smooth = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": 1,
                                "smooth_par": {"x": xs, "y": [0, 0.5, 0, -0.5, 0]}}} 

add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}
add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth"]}}

io.debug=0

In [4]:
reply = io.doit4ids(ids,ctrl_cmd_set_home())

NameError: name 'ctrl_cmd_set_home' is not defined

In [17]:
reply = io.doit4ids(ids,ctrl_cmd_start())

In [18]:
# define trajectories
reply = io.doit4ids(ids,homing)
reply = io.doit(smooth)


In [19]:
reply = io.doit(add_homing)

In [20]:
reply = io.doit(add_smooth)

In [21]:
reply = io.doit4ids(ids,ctrl_cmd_stop())

In [8]:
reply = io.doit4ids(ids,ctrl_cmd_start(value=0xD4))

type: CTRL_CMD
ctrl_cmd {
  type: CTRL_CMD_START
  board_id: 1
  value: 212.0
}



In [25]:
io.debug=1
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 1
gains['kd'] = 0
gains['ki'] = 0
gains['kkp'] = 5
gains['kkd'] = 0.01
gains['kki'] = 0.8
reply = io.doit(ctrl_cmd_start(bId=1,gains=gains))

type: CTRL_CMD
ctrl_cmd {
  type: CTRL_CMD_START
  board_id: 1
  gains {
    type: IMPEDANCE
    kp: 0.5
    ki: 0.0
    kd: 0.0
    kkp: 5.0
    kki: 0.8
    kkd: 0.01
  }
}



In [17]:
cmd

{'type': 'CTRL_CMD',
 'ctrl_cmd': {'type': 'CTRL_CMD_START',
  'board_id': 1,
  'value': 212,
  'gains': {'type': 'IMPEDANCE',
   'kp': 10,
   'kd': 1,
   'ki': 0,
   'kkp': 2,
   'kkd': 0.01,
   'kki': 0.8}}}